# Oscillating droplet 3D
## Test case description
For a description of the test cases and their setup, please refer to the preprint/paper.
## Parameter and error metric explanation
Mandatory and optional data columns for the CSV files providing the plot data. See the repository `README.md`
for general syntax information.

| column name | description | required |
| :-          | :-          | :-       |
| `solver` | solver name, e.g. *interFoam* | yes |
| `fluid_pairing` | name the two phases used for simulation, e.g. water-air | yes |
| `resolution` | mesh resolution as number of cells per spatial direction | yes |
| `time` | physical time of the data point in seconds | yes |
| `major_semi_axis_length` | Length of the semi-axis, largest at $t=0$ | yes |

## Usage
The notebook is prepared such that you can just hit `Kernel` ->
`Restart & Run All`. This plots the temporal evolution of the major semi-axis for all fluid/fluid pairings, resolutions and solvers, as well as the accumulated amplitude error.  
Per default, all solvers for which data is available are added to each plot. If you want to compare a specific set of solvers, remove the `'all'` entry from `solvers` below and list the solvers you want to compare, e.g.
`solvers = ['interFoam', 'Fluent']`. If you prescribe a solver subset and a solver is missing in the plots, make sure that its spelling matches the entries in the CSV file exactly.

In [ ]:
# Data directory containing CSV files
directory = 'data/hydrodynamic/oscillatingDroplet3D/'

# CSV data columns, split into metadata and data columns
index_columns = ['solver', 'fluid_pairing', 'resolution']
data_columns = ['time', 'major_semi_axis_length']

# Initial interface configuration
# NOTE: both values have to match the test case setup of the solvers
radius0 = 0.5e-3
amplitude0 = 0.05*radius0

# Solver selection
solvers = ['all']

import math
import os

# Use all CSV files in the data directory
# Explicitly set the file names here if you only want to compare a subset.
data_files = [filename for filename in os.listdir(directory) if 'csv' in filename]

In [ ]:
def compute_difference_to_reference(df, radius0=None, amplitude0=None):
    """
    Compute the difference between the computed amplitude and amplitude
    from the reference solution.
    """
    # This can be probably done much more elegant and concise manner
    df['delta'] = 0.0
    df['abs_delta'] = 0.0
    df['rel_abs_delta'] = 0.0
    
    # Initial major semi axis
    #major_semi_axis0 = radius0 + amplitude0
    
    # Generate name to index map for columns
    cmap = dict()
    for cname in df.columns:
        cmap[cname] = df.columns.get_loc(cname)
    
    for ridx in df.index:
        time = df.iloc[ridx, cmap['time']]
        a_sol = df.iloc[ridx, cmap['major_semi_axis_length']]
        a_ana = oscillation_n2_3d(time, df.iloc[ridx, cmap['fluid_pairing']], radius0=radius0, amplitude0=amplitude0)
        df.iloc[ridx, cmap['delta']] = a_sol - a_ana
        df.iloc[ridx, cmap['abs_delta']] = math.fabs(df.iloc[ridx, cmap['delta']])
        df.iloc[ridx, cmap['rel_abs_delta']] = math.fabs(df.iloc[ridx, cmap['delta']])/amplitude0
    
    return df

#--- Imports ---
from shared_functions import *
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams.update(latex_settings)

lin_df = solver_data_table(data_files, list(index_columns)+list(data_columns), data_dir=directory,
                           solver_subset=solvers)
df = compute_difference_to_reference(lin_df, radius0=radius0, amplitude0=amplitude0)

# In case of many notebook executions, e.g. for debugging:
# temporarily store dataframe with deltas and read it.
# The delta computation takes quite some time
#df.to_csv(directory + 'delta_frame.csv', index=False)
#df = pd.read_csv(directory + 'delta_frame.csv')

# Set multiindex
df.set_index(index_columns, inplace=True)
df.sort_index(inplace=True)

# Each parameter combination is used to create a separate figure
group_by_parameters = ["fluid_pairing", "resolution"]

# Marker and line style for each solver
n_marks = 10
solvers = list(df.index.unique(level='solver'))
solver_marker, solver_line = assign_styles_to_solvers(solvers)

for params, subsetdf in  df.groupby(level=group_by_parameters):
    fig, ax = plt.subplots(figsize=figureSize, dpi=DPI)
    times = []

    for solver, plotdf in subsetdf.groupby(level="solver"):
        ax.plot(plotdf["time"], plotdf["major_semi_axis_length"],
                label=solver,
                marker=solver_marker[solver], markevery=int(len(plotdf.index)/n_marks),
                linestyle=solver_line[solver])
        times = plotdf["time"]
        
    # Plot reference solution (only for water-air system, not usuable for other fluid pairings)
    if params[0] == 'water-air':
        reference = oscillation_n2_3d_series(times, params[0], radius0=radius0, amplitude0=amplitude0)
        ax.plot(times, reference, label="Reference")
        
    # Formatting
    ax.ticklabel_format(axis='x', style='scientific', scilimits=(-2,-2))
    ax.ticklabel_format(axis='y', style='scientific', scilimits=(-4,-4))
    
    ax.set_ylabel("major semi axis length [m]", fontsize=plotfontsize)
    ax.set_xlabel("time [s]", fontsize=plotfontsize)
    ax.tick_params(axis='both', labelsize=plotfontsize)
        
    ax.grid(which="both")
    ax.legend(fontsize=plotfontsize)
        
    title = title_from_parameters(group_by_parameters, params, "oscillating droplet 3D")

    figure_directory = "figures"
    if not os.path.isdir(figure_directory):
        os.mkdir(figure_directory)
    fig.savefig(os.path.join(figure_directory,"oscillatingDroplet3D_" + format_for_filename(title) + ".pdf"),
              bbox_inches="tight")
    
    ax.set_title(title, fontsize=plotfontsize)
    
plot_accumulated_errors(df, "oscillating_droplet_3D", "rel_abs_delta", ylabel="time avg. rel. amplitude error",
                       solver_marker=solver_marker, file_name_addendum='_accumulated', time_rel=True)